# Телеком — Поиск клиента с неоптимальным тарифом

**Цель проекта** - определение изменения выручки организации при переводе активных пользователей на более выгодные для них тарифы.

**Задачи проекты:**
- определение средней выручки с клиента;
- поиск активных клиентов;
- определение переплаты для активных пользователей и предложение выгодного тарифа;
- расчет изменения выручки компании при переводе клиентов на другой тариф;
- рекомендации по снижению падения выручки компании.

**Итоговый результат:**
- количество активных пользователей с переплатой на текущем тарифе, процент переплаты;
- снижение расходов клиента при переходе на более выгодный тариф;
- потери в выручке компании при переводе клиентов на другой тарифный план.

**Заказчик** - Директор коммерческого департамента телеком провайдера «Нупозвони».

**Данные**

Для работы предоставлены датасеты, которые содержат данные об использовании услуг виртуальной телефонии «Нупозвони». Клиентами «Нупозвони» являются организации, которым нужно распределять множество входящих вызовов на различных операторов, либо осуществлять исходящие вызовы силами операторов. Также операторы могут осуществлять внутренние вызовы, то есть вызовы друг между другом, которые осуществляются внутри сети виртуальной телефонии. Горизонт событий - 3 месяца.

Датасет `telecom_clients` генеральная совокупность клиентской базы:

- `user_id` — идентификатор клиента;
- `tariff_plan` — тарифный план клиента;
- `date_start` — дата регистрации клиента в сервисе.

Датасет `telecom_dataset` выборка данных по работе клиентов на тарифах компании:


- `user_id` — идентификатор клиента;
- `date` — дата статистики;
- `direction` — направление вызовов: `out` — исходящий вызов, `in` — входящий;
- `internal` — маркер внутренних и внешних вызовов;
- `operator_id` — идентификатор оператора;
- `is_missed_call` — маркер пропущенных вызовов;
- `calls_count` — количество звонков;
- `call_duration` — длительность всех звонков без учёта времени ожидания;
- `total_call_duration` — длительность всех звонков с учётом времени ожидания.

**Периодичность проведения анализа данных** - 1 раз в 3 месяца.

Данное исследование проводится на 01.12.2019г.

**Правила формирования тарифов**

Таблица `tariff_plan` содержит описание тарифов компании:

- `name` — тарифный план;
- `payment` — ежемесячная плата за тариф, р/мес;
- `incoming_call` — входящие звонки, бесплатны на всех тарифах;
- `call_network` — исходящие звонки внутри сети, включённые в тариф, мин;
- `price_over_limit` — цена за исходящие внутренние звонки сверх лимита, р/мин;
- `outgoing_call` — исходящие звонки, кроме внутренних, р/мин;
- `payment_operator` — Оплата за оператора, р/мес.

### Шаг 1. Изучение общей информации. Подготовка данных.

In [4]:
# импортируем библиотеки
<<код>>

In [5]:
# считываем данные csv-файла в датафреймы и сохраняем в переменные `dataset` и `clients`
# загрузка датасетов

Выведем первые 10 строчек датафрейма на экран для ознакомления с исследуемыми данными.

<вывод> clients.head(10)

<вывод> dataset.head(10)

Выведем основную информацию о датафреймах с помощью метода `info()`.

<вывод> clients.info()

<вывод> dataset.info()

#### Работа с пропусками

Выведем количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов `isnull()` и `sum()`.

In [ ]:
<<код>>

In [ ]:
<<код>>

Пропишем функция для подсчета количества пропусков по каждому столбцу датафрейма, пропущенные значения в процентном отношении выведем горизонтальным bar plot.

In [8]:
<<код>>

Более 15% пропусков содержит столбец с информацией об ID оператора. Проверим, какие данные стоят за пропусками в этом столбце.

In [ ]:
<<код>>

Присутстует тенденция отсутствия данных по оператору на запись о пропущенном звонке. Проверим, есть ли пропуски на принятых вызовах.

In [ ]:
<<код>>

Таким образом большиенство пропусков по столбццу с оператором приходится на пропущенные звонки, что логично, т.к. звонок не принят, нет сведений об операторе. Также есть 122 записи принятых звонков с пропуском, в этом случае можно ссылаться на ошибки в сборе данных.

Для последующего упрощения работы с данными принимается заменить `NaN` на числовое значение. Примем `0`, и в дальнейшем анализе определяем `0` в столбцах `internal` и `operator_id` как отсутствие данных.

In [ ]:
<<код>>

#### Изменение типов данных

Датасет `clients` имеет несоответствие по типу данным о начале регестрации клиента, определяемые как `object`.

Таблица `dataset` имеет неверный тип данных по дате статистики (`object`), маркеру внутренних и внешних вызовов (`object`) и идентификатору оператора (`float64`). 

Приведем типы данных в соответствие.

In [12]:
# изменяем тип данных о дате статистики на `datetime`
<<код>>

# изменяем тип данных о дате регестрации клиента на `datetime`
<<код>>

# изменяем тип данных о маркере внутренних и внешних вызовов на `bool`
<<код>>

# изменяем тип данных идентификатора оператора на `int`
<<код>>

#### Поиск дубликатов

Для исключения из датафрейма дубликатов, для начала проверим начичие явных дубликатов комбинацией методов `duplicated()` и `sum()`.

In [ ]:
<<код>>

Исключаем из датафрейма явные дубликаты.

In [15]:
<<код>>

#### Выявление аномалий в данных

Проверим представленные для исследования данные на наличие аномальных значений по длительности и количеству звонков.

Так как данные за день имеют несколько записей одного оператора, стоит сгруппировать по оператору и дате, подсчитать количество звонков, их продолжительность с ожиданием и без.

В сутки суммарная длительность звонков не может превышать 24 часа (1440 мин). Поставим ограничение на гистограмму.

In [16]:
# подготавливаем данные для изучения аномальных значений
<<код>>

# переводим продолжительность звонков в минуты
<<код>>

In [ ]:
# задаём размер сетки для графиков
<<код>>

# в таблице графиков —  три строки и два столбца, шесть ячеек
# в первой строим диаграмму размаха общей продолжительности звонков с учетом времени ожидания
<<код>>

# во второй ячейке гистограмму распределения общей продолжительности звонков с учетом времени ожидания
<<код>>

# в третьей ячейке — диаграмма размаха продолжительности звонков без учетом времени ожидания
<<код>>

# в чётвертой ячейке — гистограмма распределения продолжительности звонков без учетом времени ожидания
<<код>>

# в пятой ячейке — диаграмма размаха количества звонков на оператора
<<код>>

# в шестой ячейке — гистограмма распределения количества звонков
<<код>>

Диаграммы размаха показывают наличие аномально высоких значений, как по количеству звонков, так и по их общей продолжительности. По гистограмме распределения заметно, что длительность звонков в общем не более 300 минут,  без учета ожидания менее 250 минут. Распространенное количество звонков у оператора меньше 400.

Принимая во внимание, что оператор работает смену в 12 часов по 60 минут, то максимальное количество минут должно соответствовать 12х60=720 минутам, соответственно и количество звонков не должно превышать 720.

In [ ]:
<<код>>

Количество уникальных клиентов с высокими значениями по количеству и длительности вызовов оператора: 6

В данных присутствуют клиенты, операторы которых имеют аномально высокую продолжительность звонков, также высокое значение по количеству звонков. Определить причину возникновения по имеющимся данным не представляется возможным. Возможно технический сбой сбора информации, возможно непоследовательная работа самих операторов, что под одной учетной записью работает несколько человек. Проводить исключение аномальных значений не будем.

In [19]:
<<код>>

#### Добавление новых столбцов

Для аналица динамики во времени добавим столбец с указанием месяца даты статистики, для исследовательского анализа переведем длительность звонков в минуты.

In [20]:
# добавление столбца с месяцем
<<код>>

# перевед общей длительности звонков в минуты
<<код>>

# перевед длительности звонков без учета ожидания в минуты
<<код>>

#### Вывод по обработке данных

Таблица данных по клиентам не имеет пропусков, выборка данных по работе клиентов имеет пропуски по столбцу идентификатора оператора в 15% записей (8172) и по столбцу маркера внутренних и внешних вызовов менее 1% записей (пропуски в 117), пропуски принято заменить на значение "0". Изменен тип данных по столбцам даты регистрации клиента в сервисе и даты статистики на `datetime`. Изменен тип данных для столбца маркера внутренних и внешних вызовов на `bool` и идентификатора оператора на `int`.

Найдены явные дубликаты в данных и аномальные значения по длительности звонков и количеству звонков. Из датасета для исследования исключены записи дубликатов и аномальных значений.

Добавлен столбца с месяцом в датафрейм `dataset`.

### Шаг 2. Исследовательский анализ

Для исследования данных добавим тарифный план в таблицу `dataset`.

In [21]:
# добавим к выборке данных информацию о тарифном плане из таблицы клиентов
<<код>>

#### Количество уникальных клиентов в клиентской базе и уникальные клиенты в выборке данных

In [ ]:
# количество уникальных клиентов в клиентской базе
<<код>>

# количество уникальных клиентов в выборке данных
<<код>>

# количество уникальных операторов в выборке данных
<<код>>

Количество уникальных клиентов в выборке, значительно меньше клиентов по базе данным. Можно предлположить, что некоторые клиенты еще не начали пользоваться услугами, либо у нас не полная выборка по всем клиентам.

#### Количество уникальных клиентов в разбивке по дням и месяцам

In [ ]:
# подготавливаем таблицу количества клиентов в месяц, учитывая только уникальные id
<<код>>

# строим столбчатую диаграмму 
<<код>>

In [ ]:
# подготавливаем таблицу сгруппированную по дням и тарифному плану с расчетом уникальных клиентов
<<код>>

# строим линейный график изменения количества клиентов
<<код>>

Количество уникальных клиентов растет с августа по сентябрь, а в ноябре снизилось на 12 клиентов. В рассмотрении динамики притока клиентов по разным тарифам заметны высокие темпы по тарифу `С` и `B`. Клиенты на тариф `A` приходят медленнее, их количество практически стабильно последние два месяца.

#### Количество уникальных операторов в разбивке по дням и месяцам

In [ ]:
# подготавливаем таблицу количества операторов в месяц, учитывая только уникальные id
<<код>>

# строим столбчатую диаграмму 
<<код>>

In [ ]:
# подготавливаем таблицу сгруппированную по дням и тарифному плану с расчетом уникальных операторов
<<код>>

# строим линейный график изменения количества операторов
<<код>>

С операторами ситуация более позитивная, их количество растет с каждым днем до даты проведения исследования.

В динамике по дням заметны необычные скачки по одному двум дням, возможно какие-то клиенты работают с определенной периодичностью, возможно в какие-то из дней не проводилась статистика по некоторым клиентам.

Несмотря на то, что уникальное количество пользователей на тарифе А в последние месяцы стабильно, количество уникальных операторов растет. А к концу периода количество операторов, работающих на каждом тарифе примерно одинаково, хотя распределение по клиентам различается.

#### Динамика длительности звонков 

Посмотрим на активность клиентов разных тарифов, для этого представим график изменения средней длительности звонка.

In [ ]:
# подготавливаем таблицу, добваляем столбец с продолжительностью одного звонка на тарифе
<<код>>

# строим линейный график изменения количества операторов
<<код>>

Проверим, аналогична ли ситуация, если не учитывать время ожидания.

In [ ]:
# подготавливаем таблицу, добваляем столбец с продолжительностью одного звонка на тарифе без учета ожидания
<<код>>

# строим линейный график изменения количества операторов
<<код>>

Посмотрим на динамику исходящих звонков.

In [ ]:
# подготавливаем таблицу
<<код>>

# строим линейный график изменения количества операторов
<<код>>

Динамика продолжительности звонков с ожиданием и без аналогична, при том что длительность звонков без учета ожидания меньше. Изначально длительность увеличивалась, но с сентября приблизительно на одном уровне. Наибольшая продолжительность наблюдается по тарифу `B`, затем `C` и `A`. К моменту проведения исследования средняя продолжительность на всех тарифах близка к одному значению. Продолжительность исходящих звонков на протяжении всего периода наблюдения по тарифу `B` превышает другие тарифы, за исключением пиковых значений в конце августа и начале сентября по тарифам `C` и `A`.

#### Выделение направления работы колл-центров по количеству входящих и исходящих звонков

Согласно тарифным планам, входящие звонки не оплачиваются, в силу этого для компании выгоднее иметь клиентов, совершающих исходящие звонки. Рассмотрим распределение звонков по направлению. Для начала рассчитаем общее количество звонков в каждом направлении по всем клиентам.

In [ ]:
# группируем таблицу сводную таблицу по направлению и посчитаем количество звонков
<<код>>

# строим круговую диаграмму 
<<код>>

75% от всех звонков является исходящими. Рассмотрим статитстику в разрезе каждокого клиента, сгруппируем клиентов компании по преобладающему направлению звонков, совершенных за наблюдаемый период.

In [31]:
# создаем пустую таблицу, для добавления в нее уникального клиента и преимущественного направления звонков
<<код>>

# таблица для клиентов с одним направлением звонков
<<код>>

# циклом перебираем уникальных клиентов
<<код>>

    # создаем таблицу для клиента с группировкой по направлению звонков и суммированием количества звонков, 
    # сортировка по алфавиту
    <<код>>
    
    # условие для разделения клиентов, у которых два направления или одно
    <<код>>
        
        # сравниваем количество исходящих и входящих звонков
        <<код>>
           
           # если больше входящих в таблицу добавляем запись с id клиента и направление in
           <<код>>
           # если больше исходящих в таблицу добавляем запись с id клиента и направление out
           <<код>>
        # для клиентов с одним направление в переменную dir  выносим направление звонка и вносим запись в таблицу
        <<код>>

In [ ]:
# группируем таблицу сводную таблицу по клиентам и направлению звонков
<<код>>

# строим круговую диаграмму для визуализации распределения клиентов по преобладающему типу звонков
<<код>>

В рассмотрении каждого клиента индивидуально, в большинстве случаев преобладает количество входящих звонков. Возможно есть клиенты, получающие исключительно входящие звонки. Выведем `id` клиентов, операторы которых только принимают входящие звонки.

In [ ]:
# выведем id клиентов с одним направлением звонков
<<код>>

Итого 85 клиентов компании только принимают звонки, и не совершают исходящих, даже внутри сети, звонков.

#### Количество клиентов в разбивке по тарифам компании

In [ ]:
# группируем таблицу сводную таблицу по тарифному плану и посчитаем количество уникальных клиентов
<<код>>

# строим круговую диаграмму 
<<код>>

Чем дешевле тариф, тем больше клиентов его использует. Так тариф `С` имеет более половины клиентов, а тариф `А` только 11%.

#### Количество опператоров в разбивке по тарифам компании 

In [ ]:
# группируем таблицу сводную таблицу по тарифному плану и посчитаем количество уникальных операторов
<<код>>

# строим круговую диаграмму 
<<код>>

Анализ операторов более интересен, так как здесь уже не наблюдается распределение в зависимости от стоимости тарифа, количество операторов на тарифах отличается на 8 % максимум. Исходя из того, что компаний с тарифом `А` много меньше, чем на тарифе `В` или `С`, то они имеют большее количество операторов. 

#### Среднее количество операторов у клиентов с разными тарифами

In [ ]:
# создадим таблицу, сгруппированную по тарифам и клиентам с рассчетом их операторов
<<код>>

# строим столбчатую диаграмму 
<<код>>

Вывод из предыдущего действия подтвержден, количество операторов в компаниях с тарифом `А` (10) превышает количество операторов с другими тарифами (4-3).

#### Среднее количество и продолжительность исходящих внутренних звонков у клиентов с разными тарифами

Средие показатели будем рассчитывать по работе операторов в месяц.

In [ ]:
# создадим таблицу, сгруппировав по тарифам, клиентам и операторам, 
# с расчетом количества звонков и средней продолжительности звонков

<<код>>

In [ ]:
# график средних показателей по исходящим внутренним звонкам оператора в месяц
<<код>>

Оценивая количество внутренних звонков, можно считать, что отличий в работе операторов по тарифам нет. А продолжительность этих звонков по тарифу тем больше, чем больше лимит в тарифе, т.е. наиболее продолжительные разговоры ведут по тарифу `А`, наименее по тарифу `С`.

#### Среднее количество и продолжительность исходящих внешних звонков у клиентов с разными тарифами

In [ ]:
# создадим таблицу, сгруппировав по тарифам, клиентам и операторам, 
#с расчетом количества звонков и средней продолжительности звонков

<<код>>

In [ ]:
# график средних показателей по исходящим внешним звонкам оператора в месяц
<<код>>

Количество исходящих звонков вне сети также не имеет явных лидеров среди тарифа. Можно отметить низкую продолжительность звонка по тарифу `В` и много первышающую среднюю продолжительность по тарифу `A`.

#### Вывод исследовательского анализа

Для исследования представлена база с уникальными 732 клиентами, выборка данных по 307 клиентам и 1093 их отператоров. В течении периода наблюдения замечен рост количества уникальных клиентов с августа по сентябрь, к ноябрю выявлен спад количества клиентов. По операторам идет рост уникальных кадров практически с каждом днем до дня исследования. Замечена тенденция отсутствия данных статистики по клиентам с некой периодичностью, в следствие чего может быть сделано предположение о непостоянной роботе клиентов, либо некорректном сборе данных.

Более половины клиентов имеют преобладание входящих звонков над исходящими. В целом исходящих звонков больше, чем входящих. Из всех клиентов компании 85 не совершают исходящих звонков.

### Шаг 3. Расчет получаемой выручки компании

Для расчета выручки, необходимо подготовить таблицу, которая будет содержать помесячные данные о потреблении услуг клиентами.

In [41]:
# напишем функцию для сбора данных по каждому клиенту в отдельную таблицу
# для работы функции на ввод подаются: таблица клиентской базы, датасет с выборкой статистики постребления услуг, 
#данные о тарифном плане
<<код>>

    # пустая таблица для записи данных о каждом клиенте
    <<код>>
    
    # объединение выборки данных с клиентской базой
    <<код>>
    
    # цикл для рассмотрения каждого клиента
    <<код>>
        
        # таблица с данными клиента за весь период
        <<код>>

        # цикл для рассмотрения каждого месяца
        <<код>>
            
            
            <<код>> # таблица с данными клиента за месяц для расчета платы            
            <<код>> # определение тарифа клиента            
            <<код>> # данные о тарифе клиента                    
            <<код>> # количество операторов в месяц j
            
            # исключение из таблицы входящих звонков, т.к. они бесплатны
            <<код>>

            # длительность исходящих звонков внутри сети в минутах за месяц j
            <<код>>
            # длительность исходящих звонков вне сети в минутах за месяц j
            <<код>>

            <<код>> # ежемесячная оплата тарифа
            <<код>> # доплата за каждого оператора
            <<код>> # оплата исходящих звонков вне сети
                        
            #условием проверяем перерасход лимита исходящих звонков внутри сети и добавляем результат к выручке за месяц
            <<код>>
            
            # дабавляем в таблицу запись об используемых услугах за месяц
            <<код>>
    # перевод типа данных в соответствующий вид
    <<код>>

Применим функцию к исследуемым данным.

In [42]:
<<код>>

In [43]:
# выведем первые записи получившейся таблицы
<<код>>

#### Выручка компании по месяцам

In [ ]:
# таблица для граффика месячной выручки
<<код>>
# строим столбчатую диаграмму 
<<код>>

Выручка компании растет с каждым месяцем. Быстрый подъем с августа к сентябрю может быть обусловлен началом работы в августе и активным приростом новых клиентов первым месяц. В ноябре выручка снизилась, как раз возможно из-за ухода некоторых клиентов, исходя из исследовательского анализа. За весь период выручка компании составила 2 210 457.

#### Средняя месячная выручка компании

In [ ]:
<<код>>

Средняя месячная выручка за весь период: 552614.25

Также в связи с меньшим доходом по августу средняя выручка за период данных статистики много больше августовского показателя, и меньше ноябрьского. Посчитаем среднюю выручку за более стабильные месяцы с сентября по ноябрь.

In [ ]:
# cредняя месячная выручка за три месяца
<<код>>

Средняя выручка за период с сентября по ноябрь точнее характеризует работу компании, среднее значение в 664 261 уже ближе к выручке, получаемой последние месяцы.

#### Выручка компании в разбивке по тарифам

In [ ]:
# таблица для граффика месячной выручки
<<код>>

# строим столбчатую диаграмму 
<<код>>

In [ ]:
# строим круговую диаграмму 
<<код>>

39% прибыли компания получает по клиентам по тарифу `C`, 36% приходится на тариф `В` и 24% на тарифный план `А`.

In [ ]:
# подготовим таблицу для изучения месячной динамики средней платы по тарифу
<<код>>

# строим столбчатую диаграмму 
<<код>>

#### Вывод по анализу выручки компании

Выручка компании растет с каждым месяцем с августа к октябрю, так в августе компания заработала 217673 р, в сентябре - 487117 р, в октябре 763672 р, но в ноябре снизилась к 714995 р. 

Общая выручка за все четыре месяца составляет 2210457 р., при средней в месяц 552614,25 р, а за активные последние 3 месяца средняя выручка составляет 664261,33 р.

Больший вклад в выручку  приносят клиенты по тарифу `В` и `С`, 36% и 39% соответственно. На тариф `A` приходится 24% всей выручки.

По среднемесячным показателям большая плата соответствует клиентам с тарифом `A`, и превышает остальные в 2 раза, но при этом снижается с сентября, от 5971 до 5647 р/мес. По тарифу `В` в среднем оплата растет, от 2638 к 2877 р/мес, а по тарифу `С` последние три месяца примерно на одном уровне, 2100-2200 р/мес.

### Шаг 4. Определение активных клиентов, их доли в общей совокупности клиентов компании, исследование эффективности используемых тарифов

#### Определение количества исходящих звонков за последний месяц

Определим последний месяц сбора данных из общей таблицы  и применим фильтрацию к итоговой таблице выручки.

In [51]:
<<код>>

Соберем таблицу наиболее активных клиентов по количеству исходящих звонков за последний месяц.

In [ ]:
# таблица с количеством всех исходящих звонков по клиентам clients_last_month (clm)
<<код>>

# строим столбчатую диаграмму 
<<код>>

Проверим, настолько же активны клиенты, если принимать в расчет только исходящие звонки вне сети.

In [ ]:
# таблица с количеством всех исходящих звонков по клиентам clients_last_month_out (clm_out)
<<код>>

# строим столбчатую диаграмму 
<<код>>

Ситуация при исключении исходящих звонков внутри сети практически не изменилась, так что можно предположить, что клиенты топ-30 являются активными и выгодными для компании за счет большего количества исходящих звонков.

Выведем клиентов с наибольшей продолжительностью исходящих звонков, т.к. именно эта позиция интересует компанию, исходя из содержания тарифного плана.

In [ ]:
# таблица со средней продолжительностью исходящих звонков по клиентам в последний месяц clients_last_month_out_duration (clm_out_dur)
<<код>>

# строим столбчатую диаграмму 
<<код>>

По длительности звонков в лидерах клиенты с тарифом `С` и `B`. Из 30 клиентов с самыми длинными звонками всего 5 представителей тарифа `A`.


#### Распределение клиентов по вкладу в общую выручку компании за последний месяц, поиск наиболее выгодных клиентов для компании

In [ ]:
# таблица прибыли за последний месяц
<<код>>

# строим график топ-30 клиентов, вносимых наибольший вклад в выручку компании
<<код>>

В соответствии с количеством исходящих звонков наибольшую прибыль в компанию приносят клиенты с id 168062, 168361 и 168187, относящиеся к тарифу `A`.

#### Расчет количества пропущенных звонков на последний месяц

In [ ]:
# таблица с минимальным количеством пропущенных звонков по клиентам clients_last_month_missed (clm_missed)
<<код>>

# строим график топ-30 клиентов с минимальным количеством пропущенных звонков за последний месяц
<<код>>

In [ ]:
# среднее число пропущенных звонков за последний месяц среди 100 клиентов с наименьшим кол-вом пропущенных звонков
<<код>>

По количеству пропущенных звонков может быть некоректно рассматривать активность клиента, т.к. это количество может тесно коррелировать с общим количеством звонков, так если клиент не активен, ему поступает меньше звонков, соответственно мало пропускает.


#### Распределение клиентов по количеству операторов в последнем месяце

In [ ]:
# таблица с количеством операторов по клиентам clients_last_month_operators (clm_operators)
<<код>>

# строим график топ-30 клиентов по количеству операторов за последний месяц
<<код>>

#### Определение доли активных пользователей

Объединим подготовленные таблицы клиентов по предыдущим характеристикам и проверим сколько из них содержится во всех трех.

In [59]:
# подготовим таблицы для объединения, вынесем `user_id` из издекса
<<код>>

In [ ]:
# создадим таблицу для иллюстрации отношения выделенных активных клиентов к менее активным
<<код>>

# строим круговую диаграмму 
<<код>>

In [ ]:
# группируем таблицу сводную таблицу по тарифному плану и посчитаем количество уникальных клиентов
<<код>>

# строим круговую диаграмму 
<<код>>

In [ ]:
# группируем сводную таблицу активных клиентов по тарифному плану и посчитаем количество уникальных операторов
<<код>>

# строим круговую диаграмму 
<<код>>

Выборка данных по пользованию клиентами услуг телефонии содержит 307 клиентов, среди которых наиболее активными является 80 клиентов, что составляют 26%. Распределение по тарифам отличается от распределения общей массы клиентов, так среди активных наибольшее количество клиентов относится к тарифу `В` 50% (40 клиентов) затем `С` 31% (25 клиентов) и наименьшее по `А` 19% (15 клиентов). В рассмотрении количества операторов картина немного изменяется, здесь уже сравниваются клиенты тарифа `В` имеют 238 операторов, что составляет 42% всех активных операторов, по тарифу `А` 190 операторов (33%), 141 оператор на тарифе `С` - 25%.

#### Сравнение среднего месячного потребеления активного клиента со средним по тарифу

Для расчета среднего месячного потребления используем потребление услуг за три месяца с сентября по ноябрь, учитывая горизонт анализа в 3 месяца. Также сравнение будем проводить через среднее от активных клиентов, т.к. нет смысла сравнивать активных клиентов, имеющих большее потреблением услуг, со всеми пользователями.

In [63]:
# таблица средних показателей потребления тарифа активными клиентами за три месяца
<<код>>

In [64]:
# функция для сравнения потребления услуг клиентом со средним по его тарифу из числа активных клиентов

def difference_usage(clients, dataset, tariff_plan):

    # пустая таблица для записи клиентов превышающих средние трафики по тарифу
    <<код>>

    # перебор всех клиентов
    <<код>>

        # данные клиента за месяц    
        <<код>>


In [65]:
# применяем функцию 
<<код>>

In [ ]:
# сгруппируем данные по тарифу для расчета средней переплаты по тарифам
<<код>>

# строим столбчатую диаграмму 
<<код>>

 Таким образом клиенты с тарифом `A` в среднем переплачивают 925 р/мес, клиенты в тарифом `С` переплачивают 555 р/мес, по тарифу  `В` наименьшая переалата и составляет 413 р/мес.

#### Расчет оплаты активного клиента на другом тарифе

Для определения наиболее подходящего тарифа для клиента, рассчитываем потребляемый им трафик за период с сентября по ноябрь, 3 месяца, и определяем плату на каждом тарифе, в сравнении выбираем тариф с наименьшей платой. Для этого прописываем функцию, которой передаем потребляемые клиентом услуги за три месяца и условия тарифного плана. На выходе функция выдает таблицу с данными по каждому клиенту: тариф, плата на текущем тарифе, выгодный тариф, плана на выгодном тарифе.

In [67]:
# функция для сравнения оплаты услуг клиентом на разных тарифах

def difference_usage(df, tariff_plan):

    # пустая таблица для записи клиентов превышающих средние трафики по тарифу
    <<код>>

    # перебор всех клиентов
    <<код>>     

        # плата на тарифе А
        <<код>>
        #условием проверяем перерасход лимита исходящих звонков внутри сети и добавляем результат к выручке за месяц
        <<код>>

        # плата на тарифе В
        <<код>>

        # плата на тарифе С
        <<код>>     

        # сравнение и определение наименьшей платы          
        <<код>>
        
        # определение фактической платы
        <<код>>
        
        # добавление данных в таблицу по каждому клиенту
        <<код>>

In [68]:
# подготовим таблицу для передачи функции 
# таблица активных клиентов содержит данные о потребленных услугах за три месяца,
# количество исходящих внутренних и внешних звонков, количество операторов и текущий тариф
# важно, что для расчета общее количество операторов, т.к. расчет для периода, а не для месяца
<<код>>

In [69]:
# применяем функцию
<<код>>

In [70]:
# оставляем в таблице клиентов с измененным выгодным тарифом
<<код>>

# выводим первые 10 записей
<<код>>

In [ ]:
# выведем количество уникальных клиентов, для которых подходит другой тариф
<<код>>

Проанализиров трафик активных клиентов за три месяца и проведя расчет оплаты потребленного трафика на всех типах тарифа, для 52 клиентов из числа активных клиентов выгодным является другой тариф. 

### Шаг 5. Расчет выгоды для активных клиентов при переводе на более выгодный для них тариф

#### Расходы клиента при переходе на другой тариф с текущими показателями потребления услуг

Для удобства анализа разделим клиентов по группам смены тарифа, для начала проверим, с каких и на кикие тарифы выгодно перейти клиентам.

In [ ]:
# клиенты с тарифа `А` на тариф `В`
<<код>>

# клиенты с тарифа `А` на тариф `C`
<<код>>

# клиенты с тарифа `B` на тариф `A`
<<код>>

# клиенты с тарифа `B` на тариф `C`
<<код>>

# клиенты с тарифа `C` на тариф `A`
<<код>>

# клиенты с тарифа `C` на тариф `В`
<<код>>

Итого для 5 клиентов выгодно перейти с `A` на `B` и для 6 на `C`. Выгодным считается переход с `B` на тариф `А` для 2 клиентов и для 28 на тариф `C`. Переход с тарифа `C` на тариф `B` для 11 клиентов принесет экономию средств. Клиентов, для которых выгоднен переход с тарифа `C` на тариф `A` не нашлось. 

#### Расходы клиента при переходе на другой тариф с текущими показателями потребления услуг

Для каждой группе перехода на альтернативный тариф расчитываем % снижения платы.

**Группа перехода `А` на тариф `В`**

In [ ]:
<<код>>

При переходе с тарифа `А` на тариф `В` клиенты в среднем будут платить меньше на 21.93%

**Группа перехода `A` на тариф `C`**

In [ ]:
<<код>>

При переходе с тарифа `А` на тариф `C` клиенты в среднем будут платить меньше на 46.92%

**Группа перехода `В` на тариф `А`**

In [ ]:
<<код>>

При переходе с тарифа `В` на тариф `А` клиенты в среднем будут платить меньше на 12.67%

**Группа перехода `В` на тариф `C`**

In [ ]:
<<код>>

При переходе с тарифа `В` на тариф `C` клиенты в среднем будут платить меньше на 14.53%

**Группа перехода `C` на тариф `В`**

In [ ]:
<<код>>

При переходе с тарифа `C` на тариф `В` клиенты в среднем будут платить меньше на 10.41%

#### Выгода клиента при переходе на другой тариф

In [ ]:
# подготавливаем таблицу снижения оплаты клиентов при переходе с тарифа на тариф
<<код>>

# строим столбчатую диаграмму 
<<код>>

Наиболее выгодным является переход на другой тариф для клиентов с тарифом `A`, так для одних на тарифе `C` снизятся плата на 46,92%, для других на тарифе `В` снижается на 21.93%. Для клиентов с тарифом `В`: для одних переход на `А` будет означать снижение оплаты на 12.67%, для других переход на `C` на 14.53%. Для активных клиентов с тарифом `C` уменьшение трат на оплату услуг на тарифе `В` составит 10,41%. В среднем, при переводе клиенты будут платить на 21,29% меньше.

### Шаг 6. Определение изменения выручки компании при переводе активных клиентов на другие тарифы

Для определения изменения выручки при смене тарифа активных клиентов, следует для начала привести расчет выручки по остальным клиентам к формату выручки за три месяца. Для этого применим функцию расчета платы за три месяца, оставим столбец с клиентами и платой по текущему тарифу, так рассчитаем текущую выручку компании. Далее присоединим к таблице плату на более выгодном тарифе для активных клиентов, с сохранением текущей платы для пассивных клиентов.

In [79]:
# подготовим таблицу для передачи функции 
# таблица всех клиентов, содержит данные о потребленных услугах за три месяца,
# количество исходящих внутренних и внешних звонков, количество операторов и текущий тариф
# важно, что для расчета общее количество операторов, т.к. расчет для периода, а не для месяца
<<код>>

In [80]:
# применяем функцию для расчета общей платы каждого клиента за три месяца
<<код>>

In [81]:
# подготовим таблицу для передачи функции 
# таблица всех клиентов, содержит данные о потребленных услугах за три месяца,
# количество исходящих внутренних и внешних звонков, количество операторов и текущий тариф
# важно, что для расчета общее количество операторов, т.к. расчет для периода, а не для месяца
<<код>>

# применяем функцию для расчета общей платы каждого клиента за три месяца
<<код>>

# оставляет в таблице только столбцы с id клиента и суммой платы по текущему тарифу, переименовываем столбец с платой
<<код>>

# таблица активных клиентов,
# оставляет в таблице только столбцы с id клиента и суммой платы на лучшем тарифе, переименовываем столбец с платой 
<<код>>

# объединяем всех клиентов 
<<код>>

# получили таблицу с текущей выручкой по клиентам 
# и выручкой со сменой тарифа на более выгодный клиенту
# заполним пропуски в столбце с выручкой после смены тарифа значениями первого столбца (плата по текущему)
<<код>>

# выведем первые значения таблицы
<<код>>

#### Определение выручки компании при смене тарифного плана активных клиентов

In [ ]:
# подготавливаем таблицу общей выручки за три месяца до и после смены тарифа активных клиентов
<<код>>

# строим столбчатую диаграмму 
<<код>>

Определим также процентное соотношение получаемой выручки по тарифам.

In [ ]:
# подготовка таблиц для расчета распределения выручки компании до и после смены тарифа активных пользователей
<<код>>


# график средних показателей по исходящим внутренним звонкам оператора в месяц
<<код>>

Текущая выручка компании с августа по ноябрь составила 1 992 910, при изменении тарифа активных пользователей выручка компании составила бы 1 904 697 р. В рассмотрении распределения по тарифам после смены клиенты тарифа `A` будут приносить не 24, а 23%. Клиенты по тарифу `C` увеличат свою долю в выручке компании на 1% до 41%.

#### Расчет изменения выручки компании

In [ ]:
# изменение выручки за три месяца при переводе активных клиентов на вогодный для них тариф
<<код>>

Изменение тарифного плана активных клиентов снижает их оплату по потребляемым услугам, соответственно снижается общая выручка компании. В рассмотрении перевода 52 клиентов на альтернативный териф выручка снижается на 4,43%.

### Шаг 7. проверка статистических гипотез

Чтобы проверить гипотезы о равенстве средних двух генеральных совокупностей для зависимых (парных) выборок, применим t-критерий для парных выборок методом `scipy.stats.ttest_rel()`.

#### Гипотеза 1: Текущая общая выручка компании не отличается от выручки компании после смены тарифа активных клиентов.

Сформулируем нулевую и альтернативную гипотезы:

0. Нулевая гипотеза H₀: Выручка компании до и после смены тарифа одинакова;
1. Альтернативная гипотеза H₁: Выручка компании до смены тарифа и после отличаются.

In [85]:
# массив данных до смены тарифа
<<код>>
# массив данных после смены тарифа
<<код>>

Статистические показатели выборок

In [ ]:
# объем сравниваемых выборок
<<код>>
# средние значения выборок
<<код>>
# аномальные значания выборок
<<код>>
# дисперсия выборок
<<код>>
# распределение данных выборок
<<код>>

Объем выборки до: 303
Объем выборки после: 303

Среднее значение выборки до: 6577.26
Среднее значение выборки после: 6286.13

Максимальное и минимальное значение выборки до: (60912.0, 1300.0)
Максимальное и минимальное значение выборки после: (60912.0, 1300.0)

Дисперсия выборки до: 29468553.01
Дисперсия выборки после: 27661971.0

Распределение данных выборок смещено влево. Минимальные значения совпадают, в максимальное `после` меньше. Также среднее значение и дисперсия `после` меньше.

In [ ]:
# проводим t-тест для парных выборок

<<код>># уровень статистической значимости

<<код>>

Поскольку p-значение ( 0.00001 ) меньше 0,05, мы отвергаем нулевую гипотезу. У нас есть достаточно доказательств, чтобы сказать, что средняя выручка отличается при текущих тарифах клиента и при изменении тарифа активных клиентов.

#### Гипотеза 2: Выручка компании по активным клиентам на текущем тарифе не отличается от выручки по этим клиентам после смены тарифа.

Сформулируем нулевую и альтернативную гипотезы:

0. Нулевая гипотеза H₀: Выручка компании от активных клиентов до и после смены тарифа одинакова;
1. Альтернативная гипотеза H₁: Выручка компании от активных клиентов до смены тарифа и после отличаются.

In [88]:
# подготовим данные, выделим из таблицы только активных клиентов со сменой тарифа
<<код>>

# массив данных до смены тарифа
<<код>>
# массив данных после смены тарифа
<<код>>

Статистические показатели выборок

In [ ]:
# объем сравниваемых выборок
<<код>>
# средние значения выборок
<<код>>
# аномальные значания выборок
<<код>>
# дисперсия выборок
<<код>>
# распределение данных выборок
<<код>>

Объем выборки до: 52
Объем выборки после: 52

Среднее значение выборки до: 9336.64
Среднее значение выборки после: 7640.25

Максимальное и минимальное значение выборки до: (28658.0, 2639.5)
Максимальное и минимальное значение выборки после: (23634.8, 2165.3)

Дисперсия выборки до: 16091754.91
Дисперсия выборки после: 12508855.98

Выборка активных клиентов для смены тарифа отличается более высокими значениями по среднему, минимальному и максимальному значению. Дисперсия в несколько раз умеьшается после смены тарифа.

In [ ]:
# проводим t-тест для парных выборок

<<код>> # уровень статистической значимости

<<код>>

Поскольку p-значение ( 0.000001 ) меньше 0,05, мы отвергаем нулевую гипотезу. У нас есть достаточно доказательств, чтобы сказать, что средняя выручка по активным киентам отличается при текущих тарифах клиента и при изменении тарифа.

### Шаг 8. Выводы. Рекомендации по снижению падения выручки

Компания предлагает клиентам услуги телефонии по трем тарифам, наибольшей популярностью среди которых пользуется тариф `C`, затем идет тариф `B` и `A`. По количеству операторов разница между тарифами снижается и распределяется практически в равной доли, исключая тариф `A`, по нему меньше операторов.

Выручка компании изменяется такими же темпами, как и рост количества клиентов и операторов, использующих услуги телефонии. Так наблюдается стремительный рост выручки с августа по ноябрь. Средняя выручка за месяц за 4 месяца составляет 552614р, но если не брать в расчет разгоночный месяц август, то средняя выручка равна 664261р.

Наибольший вклад в прибыль компании приносят клиенты по тарифу `В` (37%) и `С` (39)%, наименьший по тарифу `A` (24%).

Для последующего удержания наиболее активных клиентов был проанализирован трафик потребления ими услуг телефонии и сопоставлен с тарифными планами, в результате чего было выявлено 52 активных пользователей, для которых наиболее выгодным считается другой тариф. Среднее снижение платы для них составит 21%.

Проверены статистические гипотезы о том, что выручка компании не изменится, при переводе клиентов на альтернативный тариф. Гипотезы опровергнуты. Данные оплаты клиентов за три месяца превышает плату после смены тарифа. 

При переводе активных клиентов на более выгодный для них тариф выручка компании за период в три месяца снизится на 4,43%.

Предложение о смене тарифа, следует предусмотреть для клиентов активно превосходящих средние трафики по их текущему тарифу также среди активных клиентов.

Рекомендация: разрабатывать предложение для клиентов, выгода которых при переходе на тариф составляет более 15%. 

### Шаг 9. Оформление дашборда и предоставление отчетной презентации заказчику

[Презентация проекта](https://drive.google.com/file/d/1BGWM3fgxxb-LNZhQMYqL0DFdElaqnfzv/view?usp=share_link)

[Дашборд проекта](https://public.tableau.com/views/DashboardTelecom03_02/DashboardTelecom?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)